In [44]:
# check for if info has been updated in the past 24 hours
# want to get info for a single campsite

""" 
need to populate in db: 
-> status_updates: is_open, last_checked, fire_restrictions, last_updated_site
-> weather_forecasts: forecast_json (jsonb), last_updated
"""
import argparse
import requests
import json
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import psycopg2
import pandas as pd
import os
import re


# Set up argument parser
#parser = argparse.ArgumentParser(description="Update dynamic data for a single campsite.")
#parser.add_argument("campsite_id", type=int, help="The ID of the campsite to update")

#args = parser.parse_args()
#campsite_id = args.campsite_id
campsite_id = 1

print(f"Updating campsite ID: {campsite_id}")

Updating campsite ID: 1


In [50]:
# now getting the campsite site_url from the database using the id
# link to the database
conn = psycopg2.connect(dbname="camping", user="arrowboy")
cur = conn.cursor()

# Fetch the site_url for the given campsite_id
cur.execute("SELECT site_url, latitude, longitude FROM campsites WHERE id = %s;", (campsite_id,))
result = cur.fetchone()
site_url, latitude, longitude = result

# soup parser
r = requests.get(site_url)  
soup = BeautifulSoup(r.text, "html.parser")

## getting is_open, last_upadted_site, last_updated, and fire_restrictions 

In [49]:
# getting containing div for is_open
div = soup.find("div", class_="status__heading text-bold")
if div:
    is_open = div.get_text(strip=True)
    if is_open == "Site Open":
        is_open = True
    else:
        is_open = False
else:
    is_open = None

# get the last_updated_site
# containing div
div = soup.find("div", class_="page__last-modified grid-container display-flex flex-justify-end")
if div:
    # extract the text
    last_updated_site = div.get_text(strip=True)
    # remove the "Last updated" text
    last_updated_site = last_updated_site.replace("Last updated", "").strip()
    # datetime convert
    # Remove 'st', 'nd', 'rd', 'th' from the day
    last_updated_site = re.sub(r'(\d{1,2})(st|nd|rd|th)', r'\1', last_updated_site)
    last_updated_site = datetime.strptime(last_updated_site, "%B %d, %Y")
else:
    last_updated_site = None
    
# last_updated = today with hours
last_updated = datetime.now()

# getting fire restrictions status

print(f"is_open: {is_open}")
print(f"last_updated_site: {last_updated_site}")
print(f"last_updated: {last_updated}")

# getting weather from openweather and lat long 
api_key = "7c61c03997ee3bf70f5e685ca593254d"
url = f"https://api.openweathermap.org/data/3.0/onecall?lat={latitude}&lon={longitude}&appid={api_key}&exclude=minutely&units=imperial"
r = requests.get(url)
forecast_json= r.json() 
print(forecast_json)


is_open: True
last_updated_site: 2025-07-08 00:00:00
last_updated: 2025-09-08 23:40:55.029887
{'cod': 401, 'message': 'Please note that using One Call 3.0 requires a separate subscription to the One Call by Call plan. Learn more here https://openweathermap.org/price. If you have a valid subscription to the One Call by Call plan, but still receive this error, then please see https://openweathermap.org/faq#error401 for more info.'}
